# OpenAI 외 도구 호출 에이전트(Tool Calling Agent)

OpenAI 외에도 `Anthropic`, `Google Gemini`, `Together.ai`, `Ollama`, `Mistral`과 같은 더 광범위한 공급자 구현을 지원합니다.

이번 챕터에서는 다양한 LLM 을 사용하여 도구 호출 에이전트를 생성하고 실행하는 방법을 살펴보겠습니다.

**참고 링크**

- [LangChain 공식 도큐먼트](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/tool_calling/)

In [8]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [9]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH15-Agents")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH15-Agents


In [10]:
from langchain.tools import tool
from typing import List, Dict
from langchain_teddynote.tools import GoogleNews


# 도구 정의
@tool
def search_news(query: str) -> List[Dict[str, str]]:
    """Search Google News by input keyword"""
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)


print(f"도구 이름: {search_news.name}")
print(f"도구 설명: {search_news.description}")

도구 이름: search_news
도구 설명: Search Google News by input keyword


In [11]:
# tools 정의
tools = [search_news]

## Agent 용 프롬프트 생성

- `chat_history` : 이전 대화 내용을 저장하는 변수 (멀티턴을 지원하지 않는다면, 생략 가능합니다.)
- `agent_scratchpad` : 에이전트가 임시로 저장하는 변수
- `input` : 사용자의 입력

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent

# 프롬프트 생성
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `search_news` tool for searching keyword related news.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

## Tool Calling 을 지원하는 다양한 LLM 목록

실습 진행을 위해서는 아래 내용을 설정해야 합니다.

**Anthropic**

- [Anthropic API 키 발급 관련](https://console.anthropic.com/settings/keys)
- `.env` 파일 내 `ANTHROPIC_API_KEY` 에 발급받은 키를 설정하세요

**Gemini**

- [Gemini API 키 발급 관련](https://aistudio.google.com/app/apikey?hl=ko)
- `.env` 파일 내 `GOOGLE_API_KEY` 에 발급받은 키를 설정하세요

**Together AI**

- [Together AI API 키 발급 관련](https://api.together.ai/)
- `.env` 파일 내 `TOGETHER_API_KEY` 에 발급받은 키를 설정하세요

**Ollama**

- [Ollama Tool Calling 지원 모델 리스트](https://ollama.com/search?c=tools)
- [이번 실습에 사용할 llama3.1 모델](https://ollama.com/library/llama3.1)
- 터미널 창에 `ollama pull llama3.1` 명령어를 입력하여 모델을 다운로드 받습니다.
- 이전에 Ollama 를 사용하지 않았다면, [Ollama](https://wikidocs.net/233805) 를 참고해 주세요.

langchain-ollama 설치를 한 뒤 진행해 주세요.

In [6]:
# !pip install -qU langchain-ollama==0.1.3

In [13]:
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
import os

# GPT-4o-mini
gpt = ChatOpenAI(model="gpt-4o-mini")

# Claude-3-5-sonnet
claude = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0)

# Gemini-1.5-pro-latest
gemini = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)

# Llama-3.1-70B-Instruct-Turbo
llama = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=os.environ["TOGETHER_API_KEY"],
    model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
)

# Llama-3.1
ollama = ChatOllama(model="llama3.2", temperature=0)

# Qwen2.5 7B (한글 성능 괜찮은 편)
qwen = ChatOllama(
    model="qwen2.5:latest",
)

LLM 기반으로 Agent 를 생성합니다.

In [14]:
from langchain.agents import create_tool_calling_agent

# Agent 생성
gpt_agent = create_tool_calling_agent(gpt, tools, prompt)
claude_agent = create_tool_calling_agent(claude, tools, prompt)
gemini_agent = create_tool_calling_agent(gemini, tools, prompt)
llama_agent = create_tool_calling_agent(llama, tools, prompt)
ollama_agent = create_tool_calling_agent(ollama, tools, prompt)
qwen_agent = create_tool_calling_agent(qwen, tools, prompt)

## AgentExecutor 생성 후 실행 및 결과 확인



In [15]:
from langchain.agents import AgentExecutor

# gpt_agent 실행
agent_executor = AgentExecutor(
    agent=gpt_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)

result = agent_executor.invoke({"input": "AI 투자와 관련된 뉴스를 검색해 주세요."})

print("Agent 실행 결과:")
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `search_news` with `{'query': 'AI 투자'}`


[{'url': 'https://news.google.com/rss/articles/CBMiggFBVV95cUxNcmxoYUI0ZjN1NXR3d0xoM0N1SzQ2aXFhSDZtejNOVWR0RWpQc0JiVXVNLUMyVlRCVC1pMTIyY01FNFFyVXRPeE5TMU95Qk5MMVRLZGNhb3VEcU9DMENGTnZWMlI0WE8zMmtGSWNickktTV9ONnFXdW5iRFFfSHFOMnNn0gGWAUFVX3lxTE1kSnhXQS1rdkk2TGg5MG9wZVd5R0xMRmx3Z1FIVGhEU0preXIwWG12RmVHYTMyRjBJcjdnYUhLUkYzcVIwSU9mTHFRd0pqY3AxM3lQZFdwb2c3bWgxalFuSDQ1bjA4Q0xQY2VNaXZTc2l0YXNsSFdQNTA3OGFkSEFoal9sNGRCWGZzOTBoSTdFdExZeEFKZw?oc=5', 'content': '오픈AI “AI 경쟁에서 중국 앞서려면 외부 투자·지원 규정 필요” - 조선비즈 - 조선비즈'}, {'url': 'https://news.google.com/rss/articles/CBMiiAFBVV95cUxPZVZNVi05aWxZUFpsbDB0X0dEZDlWSUdXTGhtQ2I1TDh2NWVGTENsZERoeVVVWU5Va1hLVEZCcnlyY3NIUjU1N0pZNm15OVV2X2tpWk5Gd0p4enFoSkNvNUdORWxkLWM4dUZvWGNfZXdEMEpneXkyclAwUUNvUDFVbzVXcnMyWVEw?oc=5', 'content': "세계 반도체 투자, AI 쏠림 가속...전통 수요는 '빨간불' - 글로벌이코노믹"}, {'url': 'https://news.google.com/rss/articles/CBMiTkFVX3lxTE94UXZ0cnU5V0tXVzZlS0o5ZjFLYU5Vd2RCTjR

다양한 llm을 사용하여 에이전트를 실행합니다.

다음은 입력받은 llm을 사용하여 Agent 를 생성하고 실행하여 결과를 출력하는 함수입니다.

In [16]:
def execute_agent(llm, tools, input_text, label):
    agent = create_tool_calling_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools, verbose=False)
    result = executor.invoke({"input": input_text})
    print(f"[{label}] 결과입니다.")
    if isinstance(result["output"], list) and len(result["output"]) > 0:
        for item in result["output"]:
            if "text" in item:
                print(item["text"])
    elif isinstance(result["output"], str):
        print(result["output"])
    else:
        print(result["output"])

각 llm 별로 에이전트를 생성하고 실행하여 결과를 출력합니다.

In [17]:
query = (
    "AI 투자와 관련된 뉴스를 검색하고, 결과를 Instagram 게시글 형식으로 작성해 주세요."
)

In [18]:
# gpt
execute_agent(gpt, tools, query, "gpt")

[gpt] 결과입니다.
📈✨ **AI 투자 동향** ✨📈

최근 AI 기술과 관련된 투자 소식이 속속 전해지고 있습니다! 여기 최신 뉴스를 소개합니다:

1. **오픈AI**: "AI 경쟁에서 중국을 앞서려면 외부 투자와 지원 규정이 필요하다"는 목소리가 나왔습니다. [자세히 보기](https://news.google.com/rss/articles/CBMiggFBVV95cUxNcmxoYUI0ZjN1NXR3d0xoM0N1SzQ2aXFhSDZtejNOVWR0RWpQc0JiVXVNLUMyVlRCVC1pMTIyY01FNFFyVXRPeE5TMU95Qk5MMVRLZGNhb3VEcU9DMENGTnZWMlI0WE8zMmtGSWNickktTV9ONnFXdW5iRFFfSHFOMnNn0gGWAUFVX3lxTE1kSnhXQS1rdkk2TGg5MG9wZVd5R0xMRmx3Z1FIVGhEU0preXIwWG12RmVHYTMyRjBJcjdnYUhLUkYzcVIwSU9mTHFRd0pqY3AxM3lQZFdwb2c3bWgxalFuSDQ1bjA4Q0xQY2VNaXZTc2l0YXNsSFdQNTA3OGFkSEFoal9sNGRCWGZzOTBoSTdFdExZeEFKZw?oc=5)

2. **세계 반도체 투자**: AI에 대한 집중이 가속화되며 전통적인 수요는 감소세를 보이고 있습니다. [자세히 보기](https://news.google.com/rss/articles/CBMiiAFBVV95cUxPZVZNVi05aWxZUFpsbDB0X0dEZDlWSUdXTGhtQ2I1TDh2NWVGTENsZERoeVVVWU5Va1hLVEZCcnlyY3NIUjU1N0pZNm15OVV2X2tpWk5Gd0p4enFoSkNvNUdORWxkLWM4dUZvWGNfZXdEMEpneXkyclAwUUNvUDFVbzVXcnMyWVEw?oc=5)

3. **AI 기반 MTS**: 5대 증권사가 AI 기반의 MTS 고도화를 위해 2205억원을 투자합니다. [자세히 보기](https://news.google.com/rss/articles/CBM

In [19]:
# claude
execute_agent(claude, tools, query, "claude")

[claude] 결과입니다.


검색 결과를 바탕으로 Instagram 게시글 형식으로 작성해 드리겠습니다.

---

📢 #AI투자 #테크트렌드 #금융혁신

🤖💰 AI 투자 최신 동향! 🚀

1. OpenAI: AI 경쟁력 강화를 위한 외부 투자 필요성 강조
   - 중국과의 경쟁에서 앞서기 위한 전략
   - 정부 지원 및 규정 마련 촉구

2. 반도체 산업의 AI 쏠림 현상
   - 글로벌 반도체 투자, AI 분야로 집중
   - 전통적 수요 감소로 인한 우려 존재

3. 국내 증권사들의 AI 기반 MTS 개발 가속화
   - 5대 증권사, 총 2205억원 투자
   - 고객 맞춤형 서비스 강화 목표

4. 채널코퍼레이션, AI 기술 고도화 위해 110억 투자 유치
   - AI 기반 디지털 마케팅 플랫폼 개발에 주력

5. NVIDIA CEO 젠슨 황의 조언
   - "위대한 기업에 투자하라"
   - AI 성장 궤적에 주목하는 투자 전략 제시

💡 AI 기술의 발전이 투자 시장에 미치는 영향이 커지고 있습니다. 여러분은 어떤 AI 기업에 관심이 있으신가요? 댓글로 여러분의 생각을 공유해주세요! 👇

#AIinvestment #TechInvesting #FinancialInnovation #OpenAI #Semiconductors #StockTrading #NVIDIA

---

이상으로 AI 투자 관련 뉴스를 Instagram 게시글 형식으로 작성해 보았습니다. 이 게시글은 주요 뉴스 내용을 간략하게 요약하고, 해시태그와 이모지를 사용하여 Instagram 스타일로 구성했습니다. 또한, 독자들의 참여를 유도하는 질문도 포함했습니다. 추가로 수정이나 보완이 필요한 부분이 있다면 말씀해 주세요.


In [20]:
# gemini
execute_agent(gemini, tools, query, "gemini")

[gemini] 결과입니다.
# AI 투자 관련 뉴스 업데이트!

AI 기술이 급속도로 발전하면서 투자 열풍 또한 뜨겁습니다. 최근 AI 투자 관련 주요 뉴스들을 소개합니다.


**1. 오픈AI, AI 경쟁 우위 확보 위한 투자 및 지원 규정 필요성 강조**

오픈AI는 중국과의 AI 경쟁에서 앞서기 위해서는 외부 투자 유치 및 정부 지원에 대한 명확한 규정 마련이 필요하다고 주장했습니다. 자세한 내용은 [기사 링크](https://news.google.com/rss/articles/CBMiggFBVV95cUxNcmxoYUI0ZjN1NXR3d0xoM0N1SzQ2aXFhSDZtejNOVWR0RWpQc0JiVXVNLUMyVlRCVC1pMTIyY01FNFFyVXRPeE5TMU95Qk5MMVRLZGNhb3VEcU9DMENGTnZWMlI0WE8zMmtGSWNickktTV9ONnFXdW5iRFFfSHFOMnNn0gGWAUFVX3lxTE1kSnhXQS1rdkk2TGg5MG9wZVd5R0xMRmx3Z1FIVGhEU0preXIwWG12RmVHYTMyRjBJcjdnYUhLUkYzcVIwSU9mTHFRd0pqY3AxM3lQZFdwb2c3bWgxalFuSDQ1bjA4Q0xQY2VNaXZTc2l0YXNsSFdQNTA3OGFkSEFoal9sNGRCWGZzOTBoSTdFdExZeEFKZw?oc=5) 에서 확인하세요.

**2. 세계 반도체 투자, AI 분야로 집중...전통 수요 감소 우려**

전 세계 반도체 투자가 AI 분야로 쏠리면서, 전통적인 수요 감소에 대한 우려가 제기되고 있습니다. [기사 링크](https://news.google.com/rss/articles/CBMiiAFBVV95cUxPZVZNVi05aWxZUFpsbDB0X0dEZDlWSUdXTGhtQ2I1TDh2NWVGTENsZERoeVVVWU5Va1hLVEZCcnlyY3NIUjU1N0pZNm15OVV2X2tpWk5Gd0p4enFoSkNvNUdORWxkLWM4dUZvWGNfZXdEMEpneXkyclAwUUNvUDFVbz

In [21]:
# llama3.1 70B (Together.ai)
execute_agent(
    llama,
    tools,
    "Search AI related news and write it in Instagram post format. Answer in Korean.",
    "llama3.1 70B",
)

[llama3.1 70B] 결과입니다.
"AI 관련 뉴스

에이모, ‘2025 Emerging AI+X TOP 100 기업’ 선정 - 테크42
OpenAI, 중국 최고의 AI 정책 제안 공개: '이것은 미국이 이길 수 있고 반드시 승리해야 하는 경주' - 시사복지
에이모, ‘2025 Emerging AI+X TOP 100’ 선정 - 벤처스퀘어
WEF “AI, 은행원·단순사무직 없애고 육체노동자 증가 시킬 것” - AI타임스
하이, CES 2025에서 AI 활용 디지털 헬스케어 진단 서비스 호평 - 메디포뉴스

#AI #AI관련뉴스 #에이모 #OpenAI #WEF #CES2025"


In [23]:
# llama3.1 8B (ollama)
execute_agent(ollama, tools, query, "llama3.2(Ollama)")

[llama3.2(Ollama)] 결과입니다.
AI 투자와 관련된 뉴스입니다.

* 오픈AI의 'AI 청사진' 공개, "AI는 전기와 같은 인프라" - 테크42
* 서울대 기술지주, 2024년 투자 성과 발표 - 스타트업엔(StartupN)
* Interactive Brokers Launches AI-Powered News Summaries for Smarter, Faster Investment Decisions - 세계뉴스통신
* Clear 5G insensitivity… Reasons for AI investment - IT조선
* DWF Labs Invests in 'TAOCAT', Selected by Binance Alpha - 블루밍비트

이 뉴스들은 AI 투자와 관련된 다양한 정보를 제공합니다. 오픈AI의 'AI 청사진' 공개는 AI가 인프라로 발전할 수 있는 가능성을 제시합니다. 서울대 기술지주는 2024년 투자 성과 발표를 통해 AI 투자의 성과를 보여주고 있습니다. Interactive Brokers는 AI-Powered News Summaries를 출시하여 투자 quyết định에 도움을 줄 수 있습니다. Clear 5G는 AI 투자의 이유로 제시되며, DWF Labs는 'TAOCAT'에 투자를 지원합니다.

이 뉴스들은 AI 투자와 관련된 다양한 정보를 제공하며, 투자자들에게 유용한 정보를 제공할 수 있습니다.


In [25]:
# qwen2.5 7B (ollama)
query = "AI 투자와 관련된 뉴스를 검색하고, 결과를 Instagram 게시글 형식으로 작성해 주세요. 한글로 답변하세요!"

execute_agent(qwen, tools, query, "qwen2.5(Ollama)")

KeyboardInterrupt: 